# Imports

In [6]:
import cs7641assn4 as a4
import numpy as np
import pandas as pd
import warnings


pd.set_option('display.max_columns', 35)
# pd.reset_option("display.max_columns")

# Establish Environment

In [ ]:
rH = -1 #-5 # reward for H(ole)
rG = 1 # 10 # reward for G(oal)
rF = -0.2# reward includes S(tart) and F(rozen)
size = 4 # height and width of square gridworld, [4, 8, 16] are included in cs7641assn4.py 
p = 0.8 # if generating a random map probability that a grid will be F(rozen)
map_name = 'x'.join([str(size)]*2) # None, if you want a random map
desc = a4.MAPS[map_name] # None, if you want a random map
is_slippery = False


epsilon = 1e-8 # convergence threshold for policy/value iteration
gamma = 0.8 # discount parameter for past policy/value iterations
max_iter = 10000 # maximum iterations for slowly converging policy/value iteration 

# Qlearning(env, rH=0, rG=1, rF=0, qepsilon=0.1, lr=0.8, gamma=0.95, episodes=10000)
qepsilon = 0.1 # epsilon value for the Q-learning epsilon greedy strategy
lr = 0.8 # Q-learning rate
qgamma = 0.95 # Q-Learning discount factor
episodes = 10000 # number of Q-learning episodes
initial = 0 # value to initialize the Q grid
decay = True

# Printing options
report = True # For cs7641assn4.py policy and value iteration functions
display_print = True # For this script

In [9]:
# Create Environment
env = a4.getEnv(env_id='hw4-FrozenLake-v0', rH=rH, rG=rG, rF=rF, 
                desc=desc,  
                is_slippery=is_slippery, render_initial=True)

# Store a representation of the map
env_desc = env.desc.astype('<U8')

# Store a representation of the state rewards
env_rs = a4.getStateReward(env)

if display_print:
    # Display reward at each state
    print('\n--Reward Values at Each State--')
    a4.matprint(a4.print_value(env_rs, width=size, height=size))

--Board--

S  F  F  F
F  H  F  H
F  F  F  H
H  F  F  G

--Actions for Position to the Left of the Goal--
{0: [(1.0, 13, -0.2, False)],
 1: [(1.0, 14, -0.2, False)],
 2: [(1.0, 15, 1, True)],
 3: [(1.0, 10, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  
-0.2    -1  -0.2    -1  
-0.2  -0.2  -0.2    -1  
  -1  -0.2  -0.2     1  


# Policy Iteration

In [5]:
pi_time = %timeit -o a4.policy_iteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=False)

116 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
pi_V, pi_policy, pi_epochs = a4.policy_iteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=True)



pi_policy_arrows = a4.print_policy(pi_policy, width=size, height=size)

if display_print:
    # Display values
    a4.matprint(a4.print_value(pi_V, width=size, height=size))

    # Display policy
    a4.matprint(pi_policy_arrows)

Policy iteration converged after  19 epochs
-0.9907  -0.9884  -0.9855  -0.9819  -0.9773  -0.9717       -5       -5  -0.9447  -0.9308  -0.9135  -0.9308  -0.9447  -0.9557  -0.9646  -0.9717  
-0.9884  -0.9855  -0.9819  -0.9773       -5  -0.9646  -0.9557       -5  -0.9308  -0.9135  -0.8919  -0.9135  -0.9308  -0.9447       -5  -0.9646  
-0.9907       -5  -0.9773  -0.9717  -0.9646  -0.9557  -0.9447       -5  -0.9135       -5  -0.8649       -5  -0.9135  -0.9308  -0.9447  -0.9557  
     -5  -0.9773  -0.9717       -5  -0.9557  -0.9447  -0.9308  -0.9135  -0.8919  -0.8649  -0.8311  -0.8649  -0.8919       -5  -0.9557       -5  
-0.9773       -5  -0.9646  -0.9557  -0.9447  -0.9308       -5       -5  -0.8649  -0.8311  -0.7889       -5       -5       -1       -5       -1  
-0.9717       -5  -0.9557  -0.9447       -5  -0.9135       -5  -0.8649  -0.8311  -0.7889  -0.7361       -5       -1       -1       -1       -1  
-0.9646  -0.9557       -5  -0.9308  -0.9135  -0.8919  -0.8649       -5  -0.7889  -0.73

# Value Iteration

In [7]:
vi_time = %timeit -o a4.valueIteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=False)

113 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
vi_V, vi_epochs = a4.valueIteration(env, epsilon=epsilon, gamma=gamma, max_iter=max_iter, report=True)



vi_policy = a4.value_to_policy(env, V=vi_V, gamma=gamma)

vi_policy_arrows = a4.print_policy(vi_policy, width=size, height=size)

if display_print:
    # display value function:
    a4.matprint(a4.print_value(vi_V, width=size, height=size))
    # display policy
    a4.matprint(vi_policy_arrows)

Value iteration converged after  97 epochs
-0.9907  -0.9884  -0.9855  -0.9819  -0.9773  -0.9717       -5       -5  -0.9447  -0.9308  -0.9135  -0.9308  -0.9447  -0.9557  -0.9646  -0.9717  
-0.9884  -0.9855  -0.9819  -0.9773       -5  -0.9646  -0.9557       -5  -0.9308  -0.9135  -0.8919  -0.9135  -0.9308  -0.9447       -5  -0.9646  
-0.9907       -5  -0.9773  -0.9717  -0.9646  -0.9557  -0.9447       -5  -0.9135       -5  -0.8649       -5  -0.9135  -0.9308  -0.9447  -0.9557  
     -5  -0.9773  -0.9717       -5  -0.9557  -0.9447  -0.9308  -0.9135  -0.8919  -0.8649  -0.8311  -0.8649  -0.8919       -5  -0.9557       -5  
-0.9773       -5  -0.9646  -0.9557  -0.9447  -0.9308       -5       -5  -0.8649  -0.8311  -0.7889       -5       -5       -1       -5       -1  
-0.9717       -5  -0.9557  -0.9447       -5  -0.9135       -5  -0.8649  -0.8311  -0.7889  -0.7361       -5       -1       -1       -1       -1  
-0.9646  -0.9557       -5  -0.9308  -0.9135  -0.8919  -0.8649       -5  -0.7889  -0.736

# Q-Learning

In [5]:
Q_time = %timeit -o a4.Qlearning(env, qepsilon, lr, qgamma, episodes=10000, initial=initial, decay=decay)

9.68 ms ± 568 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
Q, Q_epochs = a4.Qlearning(env, qepsilon, lr, qgamma, episodes)

maxQ = np.max(Q,axis=1)

Q_policy = a4.Q_to_policy(Q)

Q_policy_arrows = a4.print_policy(Q_policy, width=size, height=size)

if display_print: 
    print('--Q with all options--')
    a4.matprint(Q)
    print('\n--argmax(Q) in grid order--')
    a4.matprint(a4.print_value(maxQ, width=size, height=size))
    print('\n--Policy Matrix--')
    a4.matprint(Q_policy_arrows)

--Q with all options--
 0.273189  0.603993  -0.769452   0.135425  
-0.821506     -1.56  -0.686167  -0.738509  
-0.621164  0.299704  -0.602921  -0.533545  
 -0.61056    -1.872  -0.602614  -0.602614  
 0.143375  0.846311    -1.9344  -0.571587  
       -1        -1         -1         -1  
    -1.56   1.18565      -1.56  -0.573727  
       -1        -1         -1         -1  
-0.461056     -1.56    1.10138  -0.507187  
-0.428196   1.36987      -0.32     -1.872  
-0.470784   1.64941      -1.56      -0.16  
       -1        -1         -1         -1  
       -1        -1         -1         -1  
   -1.872   -0.3136     1.6525   0.824785  
    -0.16     -0.16       1.95    1.08416  
        1         1          1          1  

--argmax(Q) in grid order--
 0.604  -0.6862  0.2997  -0.6026  
0.8463       -1  1.1856       -1  
1.1014   1.3699  1.6494       -1  
    -1   1.6525    1.95        1  

--Policy Matrix--
↓  →  ↓  →  
↓  ←  ↓  ←  
→  ↓  ↓  ←  
←  →  →  ←  


# Notes

Default rewards in OpenAI gym Frozen-Lake-v0 are 1 for the G(oal) and 0 for everything else.

Maps are drawn according to the following logic

```
if desc and map_name are None, 
   then a default random map is drawn with 8
        using frozen_lake.generate_random_map(size=8, p=0.8)
elif desc is None and a map_name is given
   then a map_name is either '4x4' or '8x8'
        and is drawn from the dict MAPS in frozen_lake.py
elif desc is given
   then it must be in the form of a list with 
```

Default action probabilities are 1/3 chosen action, 1/3 each for right angles to chosen action, and 0 for reverse of chosen action. This is set with `is_slippery=True`. If `is_slippery=False`, then P=1 for chosen action and 0 for all other actions.

|ACTION|Value|Symbol|
|------|-----|------|
|LEFT  | 0   | ←    |
|DOWN  | 1   | ↓    |
|RIGHT | 2   | →    |
|UP    | 3   | ↑    |

# Sources

- Environment: <https://gym.openai.com/envs/FrozenLake-v0/>
- Code: <https://github.com/Twice22/HandsOnRL>
- Tutorial: <https://twice22.github.io/>

# Save Results to DataFrame

In [ ]:
results = pd.DataFrame({'rH': [rH], 
                        'rG': [rG], 
                        'rF': [rF], 
                        'size': [size], 
                        'p': [p], 
                        'desc': [desc], 
                        'map_name': [map_name],                        
                        'is_slippery': [is_slippery],
                        'epsilon': [epsilon],
                        'gamma': [gamma], 
                        'max_iter': [max_iter], 
                        'qepsilon': [qepsilon], 
                        'lr': [lr], 
                        'qgamma': [qgamma], 
                        'episodes': [episodes], 
                        'initial': [initial],
                        'env_desc': [env_desc],
                        'env_rs': [env_rs],
                        'pi_time': [pi_time.average],
                        'pi_V': [pi_V],
                        'pi_epochs': [pi_epochs],
                        'pi_policy': [pi_policy],
                        'pi_policy_arrows': [pi_policy_arrows],
                        'vi_time': [vi_time.average],
                        'vi_V': [vi_V],
                        'vi_epochs': [vi_epochs],
                        'vi_policy': [vi_policy],
                        'vi_policy_arrows': [vi_policy_arrows],
                        'Q_time': [Q_time.average],
                        'Q': [Q],
                        'Q_epochs': [Q_epochs],
                        'Q_V': [maxQ],
                        'Q_policy': [Q_policy],
                        'Q_policy_arrows': [Q_policy_arrows]})

In [14]:
if display_print: 
    display(results)

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,16,0.8,"[SFFFFFHHFFFFFFFF, FFFFHFFHFFFFFFHF, FHFFFFFHF...",16x16,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,0,"[[S, F, F, F, F, F, H, H, F, F, F, F, F, F, F,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -1....",0.116077,"[-0.99071544970536, -0.9883943121316999, -0.98...",19,"[1, 1, 1, 1, 2, 1, 0, 0, 2, 1, 1, 0, 0, 0, 0, ...","[[↓, ↓, ↓, ↓, →, ↓, ←, ←, →, ↓, ↓, ←, ←, ←, ←,...",0.112625,"[-0.9907154521919761, -0.9883943146183161, -0....",97,"[1, 1, 1, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, ...","[[↓, ↓, ↓, ↓, →, ↓, ←, ←, ↓, ↓, ↓, ↓, ↓, ↓, ←,...",0.130254,"[[-2.206321221746009, -2.1498576191789085, -2....","[-2.1498576191789085, -2.052499462302082, -1.9...","[1, 1, 2, 2, 3, 1, 0, 0, 2, 1, 3, 2, 2, 3, 0, ...","[[↓, ↓, →, →, ↑, ↓, ←, ←, →, ↓, ↑, →, →, ↑, ←,..."


# Save DataFrame to Disk

Uncomment the cells below to save the results of running this notebook to an HDF5 table.

I've commented them out to avoid contaminating the results saved from the `iterate` version of this notebook.

In [15]:
# warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# try:
#     dataset = pd.read_hdf('data.h5', key='dataset', mode='a')
# except FileNotFoundError:
#     results.to_hdf('data.h5', key='dataset', mode='a')
# else:
#     dataset.append(
#         other=results, 
#         ignore_index=True
#         ).to_hdf(
#         path_or_buf='data.h5', 
#         key='dataset', 
#         mode='a')

In [ ]:
# if display_print:
#     pd.read_hdf('data.h5', key='dataset', mode='a')